# **XGBoost Training**

In this notebook, we will build upon the baseline model we previously trained using ElasticNet and aim to improve the predictive performance by utilizing the **XGBoost** algorithm. XGBoost is a highly efficient and popular machine learning algorithm that performs well on both structured and unstructured data. Its strength lies in its ability to handle complex, non-linear relationships and its robustness to overfitting when properly tuned.

Also in this case, we will evaluate the model using two different datasets:
1. **Base Dataset**: This dataset contains the original features.
2. **Graph Dataset**: This dataset includes additional features derived from graph-based representations of the data.

For model training and evaluation, we will utilize **TimeSeriesSplit** as our cross-validation technique. This choice ensures that the training and validation sets respect the temporal order of the data.

To optimize the hyperparameters of the XGBoost model, we will use **Optuna**, a powerful hyperparameter optimization framework. Unlike traditional grid search, which exhaustively searches through a predefined set of hyperparameters, Optuna uses a more efficient approach by leveraging a dynamic optimization algorithm to explore the hyperparameter space. This approach is more resource-efficient and can lead to better model performance by finding optimal settings for parameters.

In [6]:
import xgboost as xgb
import optuna
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from pathlib import Path

In [7]:
PROCESSED_PATH = Path("data/processed")
RESULTS_PATH = Path("results")
MODELS_PATH = Path("models")
RESULTS_PATH.mkdir(parents=True, exist_ok=True)
MODELS_PATH.mkdir(parents=True, exist_ok=True)

In [8]:
def train_xgboost(dataset_name, df):
    print(f"Training on dataset: {dataset_name}")
    
    target_col = "stop_arrival_delay"
    df = df.sort_values(by=["month", "day_of_week", "hour"])    # Important the order for time series cross-validation
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

    tscv = TimeSeriesSplit(n_splits=5)
    
    # Define optimization function for XGBoost
    def objective_xgb_gpu(trial):
        xgb_params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "eta": trial.suggest_float("eta", 0.01, 0.3),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 1),
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            "seed": 42,
            "tree_method": "hist",
            "device": "cuda",
        }

        rmse_scores = []
        mae_scores = []
        r2_scores = []
        for train_idx, val_idx in tscv.split(X_scaled, y):
            X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = xgb.XGBRegressor(**xgb_params, n_estimators=500, early_stopping_rounds=50)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            preds = model.predict(X_val)

            rmse_scores.append(np.sqrt(mean_squared_error(y_val, preds)))
            mae_scores.append(mean_absolute_error(y_val, preds))
            r2_scores.append(r2_score(y_val, preds))

        return np.mean(rmse_scores)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective_xgb_gpu, n_trials=50)
    
    print(f"Best parameters for {dataset_name}: {study.best_params}")
    
    # Train final model with best hyperparameters
    best_model = xgb.XGBRegressor(
        objective="reg:squarederror",
        eval_metric="rmse",
        n_estimators=500,
        early_stopping_rounds=50,
        seed=42,
        tree_method="hist",
        device="cuda",
        **study.best_params,
    )
    best_model.fit(X_scaled, y, eval_set=[(X_scaled, y)], verbose=False)
    
    model_filename = MODELS_PATH / f"xgb_{dataset_name.lower()}.pkl"
    results_filename = RESULTS_PATH / f"xgb_results_{dataset_name.lower()}.csv"
    
    with open(model_filename, "wb") as f:
        pickle.dump(best_model, f)
    
    results[dataset_name] = {
        "RMSE": final_rmse,
        "MAE": final_mae,
        "R²": final_r2
    }
    
    pd.DataFrame({
        "RMSE": [final_rmse],
        "MAE": [final_mae],
        "R²": [final_r2]
    }).to_csv(results_filename, index=False)
    
    print(f"Model saved: {model_filename}")
    print(f"Results saved: {results_filename}")
    
    plt.figure(figsize=(10, 5))
    sorted_idx = np.argsort(best_model.feature_importances_)[::-1]
    plt.bar(range(len(X.columns)), best_model.feature_importances_[sorted_idx], align="center")
    plt.xticks(range(len(X.columns)), [X.columns[i] for i in sorted_idx], rotation=90)
    plt.title(f"Feature Importance - {dataset_name}")
    plt.show()

In [ ]:
datasets = {
    "Base": pd.read_parquet(PROCESSED_PATH / "final_data.parquet"),
    "Graph": pd.read_parquet(PROCESSED_PATH / "final_data_graph.parquet"),
}

for name, df in datasets.items():
    train_xgboost(name, df)

Training on dataset: Base


[I 2025-03-31 05:05:32,874] A new study created in memory with name: no-name-35f58862-f2e3-49ab-85e2-d00eeead9fa5
c:\Users\Ningo\projects\dynamic-train-delay-prediction-system\.venv\lib\site-packages\xgboost\core.py:729: UserWarning: [05:05:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
[I 2025-03-31 05:05:50,591] Trial 0 finished with value: 5.970750625017291 and parameters: {'eta': 0.22190295216304592, 'max_depth': 7, 'subsample': 0.5782696787692632, 'colsample_bytree': 0.9009048302457557, 'gamma': 0.9544283962573178, 'lambda': 2.1965363999838487

In [ ]:
results_filename = RESULTS_PATH / "xgb_results_base.csv"
results_df = pd.read_csv(results_filename)

print("Results from xgb_results_base:")
print(results_df)

results_filename = RESULTS_PATH / "xgb_results_graph.csv"
results_df = pd.read_csv(results_filename)

print("Results from xgb_results_graph:")
print(results_df)

In [ ]:
df = pd.read_parquet(PROCESSED_PATH / "final_data.parquet")
df.describe

## **Evaluation**  

Contrary to our expectations, the extra graph-based features may not add useful predictive power. In fact, they could be introducing noise rather than improving the model. We will see if this behavior is confirmed by later models.

But leaving out the comparison for a moment, let's analyze the result in general. 

As df.describe() shows at the beginning of this notebook, the target variable (`stop_arrival_delay`) has the following characteristics:  
- **Mean:** 2.95  
- **Standard deviation:** 7.73  
- **Median (50th percentile):** 1.0  
- **75th percentile:** 4.0  
- **Max:** 300.0

This means that:
- The median delay is just 1 minute, meaning most delays are small (ss already seen in data exploration).  
- The mean (2.95) is higher than the median, suggesting right-skewed distribution (some extreme delays).  
- The high standard deviation (7.73) indicates wide variability, likely due to rare but extreme delays.  

In other words, most train delays are characterized by relatively small time variations. The majority of delays are concentrated in the lower range, with occasional extreme outliers. This distribution makes RMSE an ideal metric for our model evaluation for several key reasons. In fact, transportation systems prioritize consistency and minimal deviation. By optimizing for RMSE, we minimize frequent, small prediction errors, we reduce the overall variability in delay predictions and we create a model that performs consistently across most scenarios

In our specific case, RMSE of ~2.07 means that, on average, the model's prediction error is around 2.07 minutes. Since most delays are small (0–4 minutes), an RMSE of 2.07 minutes suggests the model performs **reasonably well** in predicting common short delays. Of course, this metric alone does not tell us how the model behaves when predicting **large delays**, that we can consider outliers. 
The model may underpredict large delays, producing estimates that are closer to the mean (2.95 min) rather than accurately capturing outliers.  

Let us continue with our evaluation, and look at the values of additional metrics, this time on the entire datasets.

In [ ]:
# import pickle
# import numpy as np
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# def evaluate_model(dataset_name, df):

#     model_filename = MODELS_PATH / f"xgb_{dataset_name.lower()}.pkl"
#     with open(model_filename, "rb") as f:
#         model = pickle.load(f)
    
#     target_col = "stop_arrival_delay"
#     df = df.sort_values(by=["month", "day_of_week", "hour"])
#     X = df.drop(columns=[target_col])
#     y = df[target_col]
    
#     scaler = StandardScaler()
#     X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
    
#     y_pred = model.predict(X_scaled)
    
#     mae = mean_absolute_error(y, y_pred)
#     rmse = np.sqrt(mean_squared_error(y, y_pred))
#     r2 = r2_score(y, y_pred)
    
#     print(f"{dataset_name} Evaluation Results:")
#     print(f"MAE: {mae:.4f}")
#     print(f"RMSE: {rmse:.4f}")
#     print(f"R²: {r2:.4f}")
    
#     return {"MAE": mae, "RMSE": rmse, "R²": r2}


# results_base = evaluate_model("Base", df_base, target_col)

# results_graph = evaluate_model("Graph", df_graph, target_col)

In [ ]:
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import pickle

In [ ]:
# def evaluate_model(model_filename, dataset_name, X, y):
#     print(f"\nEvaluating model: {dataset_name}")
    
#     with open(model_filename, "rb") as f:
#         model = pickle.load(f)

#     y_pred = model.predict(X)

#     mae = mean_absolute_error(y, y_pred)
#     rmse = np.sqrt(mean_squared_error(y, y_pred))
#     r2 = r2_score(y, y_pred)

#     print(f"{dataset_name} - MAE: {mae:.4f}")
#     print(f"{dataset_name} - RMSE: {rmse:.4f}")
#     print(f"{dataset_name} - R^2 Score: {r2:.4f}")

#     return {"Dataset": dataset_name, "MAE": mae, "RMSE": rmse, "R^2": r2}

# datasets = {
#     "Base": pd.read_parquet(PROCESSED_PATH / "final_data.parquet"),
#     "Graph": pd.read_parquet(PROCESSED_PATH / "final_data_graph.parquet"),
# }

# results = []
# for name, df in datasets.items():
#     df = df.sort_values(by=["month", "day_of_week", "hour"])
#     X = df.drop(columns=[target_col])
#     y = df[target_col]
    
#     scaler = StandardScaler()
#     X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

#     model_filename = MODELS_PATH / f"xgb_{name.lower()}.pkl"
#     res = evaluate_model(model_filename, name, X_scaled, y)
#     results.append(res)

# df_results = pd.DataFrame(results)
# df_results.to_csv(RESULTS_PATH / "xgb_evaluation_metrics.csv", index=False)

# print("\Metrics saved in results/evaluation_metrics.csv")


## **Train XGB on dataset with Embeddings**

In the initial phase of experimentation, the XGBoost models were trained and evaluated on two datasets: the base dataset and an extended version enriched with handcrafted graph-derived features. Following these experiments, additional LSTM models were developed and tested, including a variant trained on **graph embeddings** generated through a Node2Vec algorithm applied to the railway network graph.

Interestingly, the results showed that the LSTM model trained on graph embeddings outperformed the other LSTM variants, indicating that the embeddings were able to capture relevant structural information from the railway network, beyond what was explicitly encoded in the handcrafted graph features.

Motivated by these findings, we decided to further investigate the contribution of graph embeddings by integrating them into the XGBoost framework. This decision is based on the hypothesis that, given XGBoost's superior performance and lower variance compared to LSTM models, combining it with the additional structural knowledge embedded in the graph embeddings may lead to further improvements in predictive accuracy without compromising efficiency and interpretability.

This experiment aims to assess whether the latent structural relationships learned through the Node2Vec embeddings can enhance XGBoost's predictive capabilities, providing a more informative feature space for the regression task.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import optuna
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
PROCESSED_PATH = Path("data/processed")
MODELS_PATH = Path("models")
RESULTS_PATH = Path("results")

In [ ]:
df = pd.read_parquet(PROCESSED_PATH / "train_data_fe.parquet")
df.head()

In [ ]:
embedding = pd.read_csv(Path("other") / "station_embeddings.csv")
embedding.head()

In [ ]:
df = df.merge(embedding, left_on="stop_name", right_on="station_name", how="left")
df.drop(columns=["station_name"], inplace=True)

df = df.sort_values(by=["train_number", "month", "day_of_week", "hour"])

df["prev_stop_departure_delay"] = df.groupby("train_number")["stop_departure_delay"].shift(1)

df.loc[df.groupby("train_number").head(1).index, "prev_stop_departure_delay"] = np.nan

df = df.drop(columns=["stop_departure_delay"])

df = df.dropna().reset_index(drop=True)

drop_cols = [
    "scheduled_departure_time", "scheduled_arrival_time", "stop_departure_time",
    "departure_station", "arrival_station", "stop_name"
]
df.drop(columns=drop_cols, inplace=True)

df["train_avg_delay"] = df.groupby("train_number")["stop_arrival_delay"].transform("mean")
df.drop(columns=["train_number"], inplace=True)
df.dropna(inplace=True)

df = df.drop(columns=["prev_stop_departure_delay"])

df.to_parquet(PROCESSED_PATH / "final_data_embeddings.parquet")

In [ ]:
from sklearn.decomposition import PCA

embedding_cols = [col for col in df.columns if col.startswith('0') or col.isdigit()]
pca = PCA(n_components=10)
embedding_pca = pca.fit_transform(df[embedding_cols])

pca_cols = [f"embed_pca_{i}" for i in range(embedding_pca.shape[1])]
df[pca_cols] = embedding_pca
df.drop(columns=embedding_cols, inplace=True)

In [ ]:
df.head()

In [ ]:
# n_splits = 5
# target_col = "stop_arrival_delay"
# tscv = TimeSeriesSplit(n_splits=n_splits)

In [ ]:
dataset_name = "Embeddings"
train_xgboost(dataset_name, df)

Evaluation embeddings

In [ ]:
results_filename = RESULTS_PATH / "xgb_results_embeddings.csv"
results_df = pd.read_csv(results_filename)

print("Results from xgb_results_embeddings:")
print(results_df)

In [ ]:
# def evaluate_model(model_filename, dataset_name, X, y):
#     print(f"\nEvaluating model: {dataset_name}")
    
#     with open(model_filename, "rb") as f:
#         model = pickle.load(f)

#     y_pred = model.predict(X)

#     mae = mean_absolute_error(y, y_pred)
#     rmse = np.sqrt(mean_squared_error(y, y_pred))
#     r2 = r2_score(y, y_pred)

#     print(f"{dataset_name} - MAE: {mae:.4f}")
#     print(f"{dataset_name} - RMSE: {rmse:.4f}")
#     print(f"{dataset_name} - R² Score: {r2:.4f}")

#     return {"Dataset": dataset_name, "MAE": mae, "RMSE": rmse, "R²": r2}

# datasets = {
#     "Base": pd.read_parquet(PROCESSED_PATH / "final_data.parquet"),
#     "Graph": pd.read_parquet(PROCESSED_PATH / "final_data_graph.parquet"),
# }

# results = []
# for name, df in datasets.items():
#     df = df.sort_values(by=["month", "day_of_week", "hour"])
#     X = df.drop(columns=[target_col])
#     y = df[target_col]
    
#     scaler = StandardScaler()
#     X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

#     model_filename = MODELS_PATH / f"xgb_{name.lower()}.pkl"
#     res = evaluate_model(model_filename, name, X_scaled, y)
#     results.append(res)

# df_results = pd.DataFrame(results)
# df_results.to_csv(RESULTS_PATH / "xgb_evaluation_metrics.csv", index=False)

# print("\Metrics saved in results/evaluation_metrics.csv")


In [ ]:
# def train_xgboost(df):
#     df = df.sort_values(by=["month", "day_of_week", "hour"])
#     X = df.drop(columns=[target_col])
#     y = df[target_col]

#     scaler = StandardScaler()
#     X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

#     def objective_xgb_gpu(trial):
#         xgb_params = {
#             "objective": "reg:squarederror",
#             "eval_metric": "rmse",
#             "eta": trial.suggest_float("eta", 0.01, 0.3),
#             "max_depth": trial.suggest_int("max_depth", 3, 10),
#             "subsample": trial.suggest_float("subsample", 0.5, 1.0),
#             "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
#             "gamma": trial.suggest_float("gamma", 0, 1),
#             "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#             "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#             "seed": 42,
#             "tree_method": "hist",
#             "device": "cuda",
#         }

#         rmse_scores = []
#         for train_idx, val_idx in tscv.split(X_scaled, y):
#             X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
#             y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#             model = xgb.XGBRegressor(**xgb_params, n_estimators=500, early_stopping_rounds=50)
#             model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
#             preds = model.predict(X_val)
#             rmse_scores.append(np.sqrt(mean_squared_error(y_val, preds)))

#         return np.mean(rmse_scores)

#     study = optuna.create_study(direction="minimize")
#     study.optimize(objective_xgb_gpu, n_trials=50)
    
#     print(f"Best parameters: {study.best_params}")
    
#     best_model = xgb.XGBRegressor(
#         objective="reg:squarederror",
#         eval_metric="rmse",
#         n_estimators=500,
#         early_stopping_rounds=50,
#         seed=42,
#         tree_method="hist",
#         device="cuda",
#         **study.best_params,
#     )
#     best_model.fit(X_scaled, y, eval_set=[(X_scaled, y)], verbose=False)
    
#     model_filename = MODELS_PATH / "xgb_embedding.pkl"
#     results_filename = RESULTS_PATH / "xgb_embedding_results.csv"
    
#     with open(model_filename, "wb") as f:
#         pickle.dump(best_model, f)
    
#     pd.DataFrame({"RMSE": [study.best_value]}).to_csv(results_filename, index=False)
#     print(f"Model saved: {model_filename}")
#     print(f"RMSE saved: {results_filename}")

#     plt.figure(figsize=(10, 5))
#     sorted_idx = np.argsort(best_model.feature_importances_)[::-1]
#     plt.bar(range(len(X.columns)), best_model.feature_importances_[sorted_idx], align="center")
#     plt.xticks(range(len(X.columns)), [X.columns[i] for i in sorted_idx], rotation=90)
#     plt.title("Feature Importance")
#     plt.show()

# def evaluate_model(model_filename, X, y):
#     print("\nEvaluating model")
#     with open(model_filename, "rb") as f:
#         model = pickle.load(f)
#     y_pred = model.predict(X)
#     mae = mean_absolute_error(y, y_pred)
#     rmse = np.sqrt(mean_squared_error(y, y_pred))
#     r2 = r2_score(y, y_pred)
#     print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")
#     return {"MAE": mae, "RMSE": rmse, "R²": r2}

In [ ]:
# train_xgboost(df)

# X = df.drop(columns=[target_col])
# y = df[target_col]
# scaler = StandardScaler()
# X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

# results = evaluate_model(MODELS_PATH / "xgb_embedding.pkl", X_scaled, y)
# pd.DataFrame([results]).to_csv(RESULTS_PATH / "xgb_embedding_evaluation_metrics.csv", index=False)
# print("Metrics saved in results/xgb_embedding_evaluation_metrics.csv")

The intuition behind integrating graph embeddings into the XGBoost model has proven to be correct. The model trained on the dataset enriched with Node2Vec embeddings achieved the best performance among all XGBoost variants tested, confirming that the structural information encoded in the embeddings effectively enhances the model's ability to predict train delays.